In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 6.8 MB/s 
     |████████████████████████████████| 596 kB 52.3 MB/s 
     |████████████████████████████████| 6.6 MB 52.7 MB/s 
     |████████████████████████████████| 101 kB 14.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast
device = torch.device("cuda")

In [ ]:
df=pd.read_csv("mixed_domain_godamit.csv",usecols=["data","label"])

In [ ]:
df1=pd.read_csv("covid_kd_val.csv",usecols=["data","label"])
df2=pd.read_csv("covid_kd_test.csv",usecols=["data","label"])

In [ ]:
val_text_cov=df1.loc[:,"data"].values
val_labels_cov=df1.loc[:,"label"].values
test_text_cov=df2.loc[:,"data"].values
test_labels_cov=df2.loc[:,"label"].values

In [ ]:
train_text, temp_text, train_labels, temp_labels = train_test_split(df.loc[:int(0.7*df.shape[0]),'data'],df.loc[:int(0.7*df.shape[0]):,'label'], 
                                                                    random_state=2018, 
                                                                    test_size=0.3, 
                                                                    stratify=df.loc[:0.7*int(df.shape[0]),'label'])

In [ ]:
model_name='bert-base-uncased'
bert = AutoModel.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
max_seq_len = 15
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
tokens_val = tokenizer.batch_encode_plus(
    val_text_cov.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
tokens_test = tokenizer.batch_encode_plus(
    test_text_cov.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

In [ ]:
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels_cov.tolist())

In [ ]:
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels_cov.tolist())

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 12
train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
val_data = TensorDataset(val_seq, val_mask, val_y)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [ ]:
for param in bert.parameters():
    param.requires_grad = False

For Gradient Reversal

In [ ]:
from torch.autograd import Function
class GradientReversalFn(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)
    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None

In [ ]:
class BERT_Arch(nn.Module):
    def __init__(self, bert):
      super(BERT_Arch, self).__init__()
      self.bert = bert 
      self.dropout = nn.Dropout(0.1)
      self.relu =  nn.ReLU()
      self.fc1 = nn.Linear(768,512)
      self.label_layer=nn.Sequential(nn.Linear(512,2),nn.LogSoftmax(dim=1))
      self.domain_layer=nn.Sequential(nn.Linear(512,2),nn.LogSoftmax(dim=1))
    def forward(self, sent_id, mask,grl_lambda=1.0):
      _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
      x = self.fc1(cls_hs)
      x = self.relu(x)
      x = self.dropout(x)
      x_reversed=GradientReversalFn.apply(x, grl_lambda)
      x=self.label_layer(x)
      x1=self.domain_layer(x_reversed)
      return x,x1

In [ ]:
model = BERT_Arch(bert)
model = model.to(device)

In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr = 1e-3)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_labels),
                                        y = train_labels                                                    
                                    )
class_weights = dict(zip(np.unique(train_labels), class_weights))


In [ ]:
li=[]
li.append(class_weights[0])
li.append(class_weights[1])

In [ ]:
weights= torch.tensor(li,dtype=torch.float)
weights = weights.to(device)
cross_entropy  = nn.NLLLoss(weight=weights) 
epochs = 10

In [ ]:
cross_entropy1  = nn.NLLLoss(weight=weights) 
cross_entropy2  = nn.NLLLoss(weight=weights) 

In [ ]:
def train():
  model.train()
  total_loss, total_accuracy = 0, 0
  total_preds=[]
  for step,batch in enumerate(train_dataloader):
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
    batch = [r.to(device) for r in batch]
    sent_id, mask, labels = batch
    model.zero_grad()        
    preds,preds1 = model(sent_id, mask)
    loss = cross_entropy(preds, labels)
    zeros_pred=torch.zeros(len(labels), dtype=torch.long).to(device)
    ones_pred=torch.ones(len(labels), dtype=torch.long).to(device)
    loss1 = cross_entropy1(preds1, zeros_pred)
    loss2 = cross_entropy2(preds1, ones_pred)
    total_loss =  loss+loss1+loss2
    total_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    preds=preds.detach().cpu().numpy()
    total_preds.append(preds)
  avg_loss = total_loss / len(train_dataloader)
  total_preds  = np.concatenate(total_preds, axis=0)
  return avg_loss, total_preds

In [ ]:
def evaluate():
  
  print("\nEvaluating...")
  model.eval()
  total_loss, total_accuracy = 0, 0
  total_preds = []
  for step,batch in enumerate(val_dataloader):
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))
    batch = [t.to(device) for t in batch]
    sent_id, mask, labels = batch
    with torch.no_grad():
      preds,_ = model(sent_id, mask)
      loss = cross_entropy(preds,labels)
      total_loss = total_loss + loss.item()
      preds = preds.detach().cpu().numpy()
      total_preds.append(preds)
  avg_loss = total_loss / len(val_dataloader) 
  total_preds  = np.concatenate(total_preds, axis=0)
  return avg_loss, total_preds

In [ ]:
best_valid_loss = float('inf')
train_losses=[]
valid_losses=[]
for epoch in range(epochs):
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    train_loss, _ = train()
    valid_loss, _ = evaluate()
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 10
  Batch    50  of  2,568.
  Batch   100  of  2,568.
  Batch   150  of  2,568.
  Batch   200  of  2,568.
  Batch   250  of  2,568.
  Batch   300  of  2,568.
  Batch   350  of  2,568.
  Batch   400  of  2,568.
  Batch   450  of  2,568.
  Batch   500  of  2,568.
  Batch   550  of  2,568.
  Batch   600  of  2,568.
  Batch   650  of  2,568.
  Batch   700  of  2,568.
  Batch   750  of  2,568.
  Batch   800  of  2,568.
  Batch   850  of  2,568.
  Batch   900  of  2,568.
  Batch   950  of  2,568.
  Batch 1,000  of  2,568.
  Batch 1,050  of  2,568.
  Batch 1,100  of  2,568.
  Batch 1,150  of  2,568.
  Batch 1,200  of  2,568.
  Batch 1,250  of  2,568.
  Batch 1,300  of  2,568.
  Batch 1,350  of  2,568.
  Batch 1,400  of  2,568.
  Batch 1,450  of  2,568.
  Batch 1,500  of  2,568.
  Batch 1,550  of  2,568.
  Batch 1,600  of  2,568.
  Batch 1,650  of  2,568.
  Batch 1,700  of  2,568.
  Batch 1,750  of  2,568.
  Batch 1,800  of  2,568.
  Batch 1,850  of  2,568.
  Batch 1,900  of  2,56

In [ ]:
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))
model1 = BERT_Arch(bert)
model1 = model1.to(device)
model1.load_state_dict(torch.load(path))
model2= BERT_Arch(bert)
model2= model2.to(device)
model2.load_state_dict(torch.load(path))
model3= BERT_Arch(bert)
model3= model3.to(device)
model3.load_state_dict(torch.load(path))
model4= BERT_Arch(bert)
model4= model4.to(device)
model4.load_state_dict(torch.load(path))

Fine-Tuning for time quantum of 1 month

In [ ]:
for i in range(1,25):
  train_text_cov=df1.loc[:int(0.0416*i*0.6*df1.shape[0]),'data'].values
  train_labels_cov=df1.loc[:int(0.0416*i*0.6*df1.shape[0]),'label'].values
  val_text_cov=df1.loc[int(0.0416*i*0.6*df1.shape[0]):int(0.0416*i*0.8*df1.shape[0]),'data'].values
  val_labels_cov=df1.loc[int(0.0416*i*0.6*df1.shape[0]):int(0.0416*i*0.8*df1.shape[0]),"label"].values
  test_text_cov=df1.loc[int(0.0416*i*0.8*df1.shape[0]):,'data'].values
  test_labels_cov=df1.loc[int(0.0416*i*0.8*df1.shape[0]):,'label'].values
  max_seq_len = 15
  tokens_train = tokenizer.batch_encode_plus(
      train_text_cov.tolist(),
      max_length = max_seq_len,
      pad_to_max_length=True,
      truncation=True,
      return_token_type_ids=False
  )
  tokens_val = tokenizer.batch_encode_plus(
    val_text_cov.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)
  tokens_test = tokenizer.batch_encode_plus(
      test_text_cov.tolist(),
      max_length = max_seq_len,
      pad_to_max_length=True,
      truncation=True,
      return_token_type_ids=False
  )
  train_seq = torch.tensor(tokens_train['input_ids'])
  train_mask = torch.tensor(tokens_train['attention_mask'])
  train_y = torch.tensor(train_labels_cov.tolist())
  val_seq = torch.tensor(tokens_val['input_ids'])
  val_mask = torch.tensor(tokens_val['attention_mask'])
  val_y = torch.tensor(val_labels_cov.tolist())
  test_seq = torch.tensor(tokens_test['input_ids'])
  test_mask = torch.tensor(tokens_test['attention_mask'])
  test_y = torch.tensor(test_labels_cov.tolist())
  batch_size = 12
  train_data = TensorDataset(train_seq, train_mask, train_y)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
  val_data = TensorDataset(val_seq, val_mask, val_y)
  val_sampler = SequentialSampler(val_data)
  val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)
  best_valid_loss = float('inf')
  train_losses=[]
  valid_losses=[]
  epochs=4
  for epoch in range(epochs):
      print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
      train_loss, _ = train()
      valid_loss, _ = evaluate()
      if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
      train_losses.append(train_loss)
      valid_losses.append(valid_loss)
      print(f'\nTraining Loss: {train_loss:.3f}')
      print(f'Validation Loss: {valid_loss:.3f}')
  path = 'saved_weights1.pt'
  model1.load_state_dict(torch.load(path))
  with torch.no_grad():
    preds,_ = model1(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()
  preds = np.argmax(preds, axis = 1)
  test_y = torch.tensor(test_labels_cov.tolist())
  print(classification_report(test_y, preds))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



 Epoch 1 / 4
  Batch    50  of    146.
  Batch   100  of    146.

Evaluating...

Training Loss: 0.015
Validation Loss: 0.688

 Epoch 2 / 4
  Batch    50  of    146.
  Batch   100  of    146.

Evaluating...

Training Loss: 0.014
Validation Loss: 0.572

 Epoch 3 / 4
  Batch    50  of    146.
  Batch   100  of    146.

Evaluating...

Training Loss: 0.014
Validation Loss: 0.536

 Epoch 4 / 4
  Batch    50  of    146.
  Batch   100  of    146.

Evaluating...

Training Loss: 0.013
Validation Loss: 0.471
              precision    recall  f1-score   support

           0       0.79      0.78      0.79      7567
           1       0.79      0.79      0.79      7634

    accuracy                           0.79     15201
   macro avg       0.79      0.79      0.79     15201
weighted avg       0.79      0.79      0.79     15201



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



 Epoch 1 / 4
  Batch    50  of    292.
  Batch   100  of    292.
  Batch   150  of    292.
  Batch   200  of    292.
  Batch   250  of    292.

Evaluating...
  Batch    50  of     98.

Training Loss: 0.008
Validation Loss: 0.494

 Epoch 2 / 4
  Batch    50  of    292.
  Batch   100  of    292.
  Batch   150  of    292.
  Batch   200  of    292.
  Batch   250  of    292.

Evaluating...
  Batch    50  of     98.

Training Loss: 0.006
Validation Loss: 0.408

 Epoch 3 / 4
  Batch    50  of    292.
  Batch   100  of    292.
  Batch   150  of    292.
  Batch   200  of    292.
  Batch   250  of    292.

Evaluating...
  Batch    50  of     98.

Training Loss: 0.007
Validation Loss: 0.376

 Epoch 4 / 4
  Batch    50  of    292.
  Batch   100  of    292.
  Batch   150  of    292.
  Batch   200  of    292.
  Batch   250  of    292.

Evaluating...
  Batch    50  of     98.

Training Loss: 0.006
Validation Loss: 0.382
              precision    recall  f1-score   support

           0       0.82  

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


...


 Epoch 1 / 4
  Batch    50  of    584.
  Batch   100  of    584.
  Batch   150  of    584.
  Batch   200  of    584.
  Batch   250  of    584.
  Batch   300  of    584.
  Batch   350  of    584.
  Batch   400  of    584.
  Batch   450  of    584.
  Batch   500  of    584.
  Batch   550  of    584.

Evaluating...
  Batch    50  of    195.
  Batch   100  of    195.
  Batch   150  of    195.

Training Loss: 0.003
Validation Loss: 0.357

 Epoch 2 / 4
  Batch    50  of    584.
  Batch   100  of    584.
  Batch   150  of    584.
  Batch   200  of    584.
  Batch   250  of    584.
  Batch   300  of    584.
  Batch   350  of    584.
  Batch   400  of    584.
  Batch   450  of    584.
  Batch   500  of    584.
  Batch   550  of    584.

Evaluating...
  Batch    50  of    195.
  Batch   100  of    195.
  Batch   150  of    195.

Training Loss: 0.003
Validation Loss: 0.341

 Epoch 3 / 4
  Batch    50  of    584.
  Batch   100  of    584.
  Batch   150  of    584.
  Batch   200  of    584.
  Batc

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



 Epoch 1 / 4
  Batch    50  of    730.
  Batch   100  of    730.
  Batch   150  of    730.
  Batch   200  of    730.
  Batch   250  of    730.
  Batch   300  of    730.
  Batch   350  of    730.
  Batch   400  of    730.
  Batch   450  of    730.
  Batch   500  of    730.
  Batch   550  of    730.
  Batch   600  of    730.
  Batch   650  of    730.
  Batch   700  of    730.

Evaluating...
  Batch    50  of    244.
  Batch   100  of    244.
  Batch   150  of    244.
  Batch   200  of    244.

Training Loss: 0.003
Validation Loss: 0.343

 Epoch 2 / 4
  Batch    50  of    730.
  Batch   100  of    730.
  Batch   150  of    730.
  Batch   200  of    730.
  Batch   250  of    730.
  Batch   300  of    730.
  Batch   350  of    730.
  Batch   400  of    730.
  Batch   450  of    730.
  Batch   500  of    730.
  Batch   550  of    730.
  Batch   600  of    730.
  Batch   650  of    730.
  Batch   700  of    730.

Evaluating...
  Batch    50  of    244.
  Batch   100  of    244.
  Batch   150

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



 Epoch 1 / 4
  Batch    50  of    876.
  Batch   100  of    876.
  Batch   150  of    876.
  Batch   200  of    876.
  Batch   250  of    876.
  Batch   300  of    876.
  Batch   350  of    876.
  Batch   400  of    876.
  Batch   450  of    876.
  Batch   500  of    876.
  Batch   550  of    876.
  Batch   600  of    876.
  Batch   650  of    876.
  Batch   700  of    876.
  Batch   750  of    876.
  Batch   800  of    876.
  Batch   850  of    876.

Evaluating...
  Batch    50  of    292.
  Batch   100  of    292.
  Batch   150  of    292.
  Batch   200  of    292.
  Batch   250  of    292.

Training Loss: 0.002
Validation Loss: 0.297

 Epoch 2 / 4
  Batch    50  of    876.
  Batch   100  of    876.
  Batch   150  of    876.
  Batch   200  of    876.
  Batch   250  of    876.
  Batch   300  of    876.
  Batch   350  of    876.
  Batch   400  of    876.
  Batch   450  of    876.
  Batch   500  of    876.
  Batch   550  of    876.
  Batch   600  of    876.
  Batch   650  of    876.
  

Fine-Tuning for time quantum of 2 months

In [ ]:
for i in range(2,26,2):
  train_text_cov=df1.loc[:int(0.0416*i*0.6*df1.shape[0]),'data'].values
  train_labels_cov=df1.loc[:int(0.0416*i*0.6*df1.shape[0]),'label'].values
  val_text_cov=df1.loc[int(0.0416*i*0.6*df1.shape[0]):int(0.0416*i*0.8*df1.shape[0]),'data'].values
  val_labels_cov=df1.loc[int(0.0416*i*0.6*df1.shape[0]):int(0.0416*i*0.8*df1.shape[0]),"label"].values
  test_text_cov=df1.loc[int(0.0416*i*0.8*df1.shape[0]):,'data'].values
  test_labels_cov=df1.loc[int(0.0416*i*0.8*df1.shape[0]):,'label'].values
  max_seq_len = 15
  tokens_train = tokenizer.batch_encode_plus(
      train_text_cov.tolist(),
      max_length = max_seq_len,
      pad_to_max_length=True,
      truncation=True,
      return_token_type_ids=False
  )
  tokens_val = tokenizer.batch_encode_plus(
    val_text_cov.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)
  tokens_test = tokenizer.batch_encode_plus(
      test_text_cov.tolist(),
      max_length = max_seq_len,
      pad_to_max_length=True,
      truncation=True,
      return_token_type_ids=False
  )
  train_seq = torch.tensor(tokens_train['input_ids'])
  train_mask = torch.tensor(tokens_train['attention_mask'])
  train_y = torch.tensor(train_labels_cov.tolist())
  val_seq = torch.tensor(tokens_val['input_ids'])
  val_mask = torch.tensor(tokens_val['attention_mask'])
  val_y = torch.tensor(val_labels_cov.tolist())
  test_seq = torch.tensor(tokens_test['input_ids'])
  test_mask = torch.tensor(tokens_test['attention_mask'])
  test_y = torch.tensor(test_labels_cov.tolist())
  batch_size = 12
  train_data = TensorDataset(train_seq, train_mask, train_y)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
  val_data = TensorDataset(val_seq, val_mask, val_y)
  val_sampler = SequentialSampler(val_data)
  val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)
  best_valid_loss = float('inf')
  train_losses=[]
  valid_losses=[]
  epochs=4
  for epoch in range(epochs):
      print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
      train_loss, _ = train()
      valid_loss, _ = evaluate()
      if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model2.state_dict(), 'saved_weights2.pt')
      train_losses.append(train_loss)
      valid_losses.append(valid_loss)
      print(f'\nTraining Loss: {train_loss:.3f}')
      print(f'Validation Loss: {valid_loss:.3f}')
  path = 'saved_weights2.pt'
  model2.load_state_dict(torch.load(path))
  with torch.no_grad():
    preds,_ = model2(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()
  preds = np.argmax(preds, axis = 1)
  test_y = torch.tensor(test_labels_cov.tolist())
  print(classification_report(test_y, preds))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



 Epoch 1 / 4
  Batch    50  of     73.

Evaluating...

Training Loss: 0.030
Validation Loss: 0.654

 Epoch 2 / 4
  Batch    50  of     73.

Evaluating...

Training Loss: 0.031
Validation Loss: 0.637

 Epoch 3 / 4
  Batch    50  of     73.

Evaluating...

Training Loss: 0.027
Validation Loss: 0.607

 Epoch 4 / 4
  Batch    50  of     73.

Evaluating...

Training Loss: 0.028
Validation Loss: 0.609
              precision    recall  f1-score   support

           0       0.66      0.69      0.68      8128
           1       0.68      0.66      0.67      8240

    accuracy                           0.67     16368
   macro avg       0.67      0.67      0.67     16368
weighted avg       0.67      0.67      0.67     16368



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



 Epoch 1 / 4
  Batch    50  of    146.
  Batch   100  of    146.

Evaluating...

Training Loss: 0.013
Validation Loss: 0.593

 Epoch 2 / 4
  Batch    50  of    146.
  Batch   100  of    146.

Evaluating...

Training Loss: 0.014
Validation Loss: 0.547

 Epoch 3 / 4
  Batch    50  of    146.
  Batch   100  of    146.

Evaluating...

Training Loss: 0.015
Validation Loss: 0.524

 Epoch 4 / 4
  Batch    50  of    146.
  Batch   100  of    146.

Evaluating...

Training Loss: 0.012
Validation Loss: 0.510
              precision    recall  f1-score   support

           0       0.76      0.74      0.75      7567
           1       0.75      0.77      0.76      7634

    accuracy                           0.75     15201
   macro avg       0.75      0.75      0.75     15201
weighted avg       0.75      0.75      0.75     15201



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


...


 Epoch 1 / 4
  Batch    50  of    730.
  Batch   100  of    730.
  Batch   150  of    730.
  Batch   200  of    730.
  Batch   250  of    730.
  Batch   300  of    730.
  Batch   350  of    730.
  Batch   400  of    730.
  Batch   450  of    730.
  Batch   500  of    730.
  Batch   550  of    730.
  Batch   600  of    730.
  Batch   650  of    730.
  Batch   700  of    730.

Evaluating...
  Batch    50  of    244.
  Batch   100  of    244.
  Batch   150  of    244.
  Batch   200  of    244.

Training Loss: 0.002
Validation Loss: 0.316

 Epoch 2 / 4
  Batch    50  of    730.
  Batch   100  of    730.
  Batch   150  of    730.
  Batch   200  of    730.
  Batch   250  of    730.
  Batch   300  of    730.
  Batch   350  of    730.
  Batch   400  of    730.
  Batch   450  of    730.
  Batch   500  of    730.
  Batch   550  of    730.
  Batch   600  of    730.
  Batch   650  of    730.
  Batch   700  of    730.

Evaluating...
  Batch    50  of    244.
  Batch   100  of    244.
  Batch   150

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



 Epoch 1 / 4
  Batch    50  of    803.
  Batch   100  of    803.
  Batch   150  of    803.
  Batch   200  of    803.
  Batch   250  of    803.
  Batch   300  of    803.
  Batch   350  of    803.
  Batch   400  of    803.
  Batch   450  of    803.
  Batch   500  of    803.
  Batch   550  of    803.
  Batch   600  of    803.
  Batch   650  of    803.
  Batch   700  of    803.
  Batch   750  of    803.
  Batch   800  of    803.

Evaluating...
  Batch    50  of    268.
  Batch   100  of    268.
  Batch   150  of    268.
  Batch   200  of    268.
  Batch   250  of    268.

Training Loss: 0.002
Validation Loss: 0.367

 Epoch 2 / 4
  Batch    50  of    803.
  Batch   100  of    803.
  Batch   150  of    803.
  Batch   200  of    803.
  Batch   250  of    803.
  Batch   300  of    803.
  Batch   350  of    803.
  Batch   400  of    803.
  Batch   450  of    803.
  Batch   500  of    803.
  Batch   550  of    803.
  Batch   600  of    803.
  Batch   650  of    803.
  Batch   700  of    803.
  

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



 Epoch 1 / 4
  Batch    50  of    876.
  Batch   100  of    876.
  Batch   150  of    876.
  Batch   200  of    876.
  Batch   250  of    876.
  Batch   300  of    876.
  Batch   350  of    876.
  Batch   400  of    876.
  Batch   450  of    876.
  Batch   500  of    876.
  Batch   550  of    876.
  Batch   600  of    876.
  Batch   650  of    876.
  Batch   700  of    876.
  Batch   750  of    876.
  Batch   800  of    876.
  Batch   850  of    876.

Evaluating...
  Batch    50  of    292.
  Batch   100  of    292.
  Batch   150  of    292.
  Batch   200  of    292.
  Batch   250  of    292.

Training Loss: 0.002
Validation Loss: 0.302

 Epoch 2 / 4
  Batch    50  of    876.
  Batch   100  of    876.
  Batch   150  of    876.
  Batch   200  of    876.
  Batch   250  of    876.
  Batch   300  of    876.
  Batch   350  of    876.
  Batch   400  of    876.
  Batch   450  of    876.
  Batch   500  of    876.
  Batch   550  of    876.
  Batch   600  of    876.
  Batch   650  of    876.
  

Fine-Tuning for time quantum of 3 months

In [ ]:
for i in range(3,27,3):
  train_text_cov=df1.loc[:int(0.0416*i*0.6*df1.shape[0]),'data'].values
  train_labels_cov=df1.loc[:int(0.0416*i*0.6*df1.shape[0]),'label'].values
  val_text_cov=df1.loc[int(0.0416*i*0.6*df1.shape[0]):int(0.0416*i*0.8*df1.shape[0]),'data'].values
  val_labels_cov=df1.loc[int(0.0416*i*0.6*df1.shape[0]):int(0.0416*i*0.8*df1.shape[0]),"label"].values
  test_text_cov=df1.loc[int(0.0416*i*0.8*df1.shape[0]):,'data'].values
  test_labels_cov=df1.loc[int(0.0416*i*0.8*df1.shape[0]):,'label'].values
  max_seq_len = 15
  tokens_train = tokenizer.batch_encode_plus(
      train_text_cov.tolist(),
      max_length = max_seq_len,
      pad_to_max_length=True,
      truncation=True,
      return_token_type_ids=False
  )
  tokens_val = tokenizer.batch_encode_plus(
    val_text_cov.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)
  tokens_test = tokenizer.batch_encode_plus(
      test_text_cov.tolist(),
      max_length = max_seq_len,
      pad_to_max_length=True,
      truncation=True,
      return_token_type_ids=False
  )
  train_seq = torch.tensor(tokens_train['input_ids'])
  train_mask = torch.tensor(tokens_train['attention_mask'])
  train_y = torch.tensor(train_labels_cov.tolist())
  val_seq = torch.tensor(tokens_val['input_ids'])
  val_mask = torch.tensor(tokens_val['attention_mask'])
  val_y = torch.tensor(val_labels_cov.tolist())
  test_seq = torch.tensor(tokens_test['input_ids'])
  test_mask = torch.tensor(tokens_test['attention_mask'])
  test_y = torch.tensor(test_labels_cov.tolist())
  batch_size = 12
  train_data = TensorDataset(train_seq, train_mask, train_y)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
  val_data = TensorDataset(val_seq, val_mask, val_y)
  val_sampler = SequentialSampler(val_data)
  val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)
  best_valid_loss = float('inf')
  train_losses=[]
  valid_losses=[]
  epochs=4
  for epoch in range(epochs):
      print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
      train_loss, _ = train()
      valid_loss, _ = evaluate()
      if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model3.state_dict(), 'saved_weights3.pt')
      train_losses.append(train_loss)
      valid_losses.append(valid_loss)
      print(f'\nTraining Loss: {train_loss:.3f}')
      print(f'Validation Loss: {valid_loss:.3f}')
  path = 'saved_weights3.pt'
  model3.load_state_dict(torch.load(path))
  with torch.no_grad():
    preds,_ = model3(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()
  preds = np.argmax(preds, axis = 1)
  test_y = torch.tensor(test_labels_cov.tolist())
  print(classification_report(test_y, preds))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



 Epoch 1 / 4
  Batch    50  of    110.
  Batch   100  of    110.

Evaluating...

Training Loss: 0.020
Validation Loss: 0.667

 Epoch 2 / 4
  Batch    50  of    110.
  Batch   100  of    110.

Evaluating...

Training Loss: 0.018
Validation Loss: 0.626

 Epoch 3 / 4
  Batch    50  of    110.
  Batch   100  of    110.

Evaluating...

Training Loss: 0.018
Validation Loss: 0.579

 Epoch 4 / 4
  Batch    50  of    110.
  Batch   100  of    110.

Evaluating...

Training Loss: 0.016
Validation Loss: 0.567
              precision    recall  f1-score   support

           0       0.83      0.58      0.68      7844
           1       0.68      0.88      0.77      7941

    accuracy                           0.73     15785
   macro avg       0.76      0.73      0.73     15785
weighted avg       0.75      0.73      0.73     15785



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



 Epoch 1 / 4
  Batch    50  of    219.
  Batch   100  of    219.
  Batch   150  of    219.
  Batch   200  of    219.

Evaluating...
  Batch    50  of     73.

Training Loss: 0.009
Validation Loss: 0.473

 Epoch 2 / 4
  Batch    50  of    219.
  Batch   100  of    219.
  Batch   150  of    219.
  Batch   200  of    219.

Evaluating...
  Batch    50  of     73.

Training Loss: 0.008
Validation Loss: 0.448

 Epoch 3 / 4
  Batch    50  of    219.
  Batch   100  of    219.
  Batch   150  of    219.
  Batch   200  of    219.

Evaluating...
  Batch    50  of     73.

Training Loss: 0.009
Validation Loss: 0.425

 Epoch 4 / 4
  Batch    50  of    219.
  Batch   100  of    219.
  Batch   150  of    219.
  Batch   200  of    219.

Evaluating...
  Batch    50  of     73.

Training Loss: 0.009
Validation Loss: 0.423
              precision    recall  f1-score   support

           0       0.88      0.72      0.79      6969
           1       0.76      0.90      0.83      7065

    accuracy        

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


...


 Epoch 1 / 4
  Batch    50  of    657.
  Batch   100  of    657.
  Batch   150  of    657.
  Batch   200  of    657.
  Batch   250  of    657.
  Batch   300  of    657.
  Batch   350  of    657.
  Batch   400  of    657.
  Batch   450  of    657.
  Batch   500  of    657.
  Batch   550  of    657.
  Batch   600  of    657.
  Batch   650  of    657.

Evaluating...
  Batch    50  of    219.
  Batch   100  of    219.
  Batch   150  of    219.
  Batch   200  of    219.

Training Loss: 0.003
Validation Loss: 0.338

 Epoch 2 / 4
  Batch    50  of    657.
  Batch   100  of    657.
  Batch   150  of    657.
  Batch   200  of    657.
  Batch   250  of    657.
  Batch   300  of    657.
  Batch   350  of    657.
  Batch   400  of    657.
  Batch   450  of    657.
  Batch   500  of    657.
  Batch   550  of    657.
  Batch   600  of    657.
  Batch   650  of    657.

Evaluating...
  Batch    50  of    219.
  Batch   100  of    219.
  Batch   150  of    219.
  Batch   200  of    219.

Training Los

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



 Epoch 1 / 4
  Batch    50  of    766.
  Batch   100  of    766.
  Batch   150  of    766.
  Batch   200  of    766.
  Batch   250  of    766.
  Batch   300  of    766.
  Batch   350  of    766.
  Batch   400  of    766.
  Batch   450  of    766.
  Batch   500  of    766.
  Batch   550  of    766.
  Batch   600  of    766.
  Batch   650  of    766.
  Batch   700  of    766.
  Batch   750  of    766.

Evaluating...
  Batch    50  of    256.
  Batch   100  of    256.
  Batch   150  of    256.
  Batch   200  of    256.
  Batch   250  of    256.

Training Loss: 0.002
Validation Loss: 0.308

 Epoch 2 / 4
  Batch    50  of    766.
  Batch   100  of    766.
  Batch   150  of    766.
  Batch   200  of    766.
  Batch   250  of    766.
  Batch   300  of    766.
  Batch   350  of    766.
  Batch   400  of    766.
  Batch   450  of    766.
  Batch   500  of    766.
  Batch   550  of    766.
  Batch   600  of    766.
  Batch   650  of    766.
  Batch   700  of    766.
  Batch   750  of    766.

E

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



 Epoch 1 / 4
  Batch    50  of    876.
  Batch   100  of    876.
  Batch   150  of    876.
  Batch   200  of    876.
  Batch   250  of    876.
  Batch   300  of    876.
  Batch   350  of    876.
  Batch   400  of    876.
  Batch   450  of    876.
  Batch   500  of    876.
  Batch   550  of    876.
  Batch   600  of    876.
  Batch   650  of    876.
  Batch   700  of    876.
  Batch   750  of    876.
  Batch   800  of    876.
  Batch   850  of    876.

Evaluating...
  Batch    50  of    292.
  Batch   100  of    292.
  Batch   150  of    292.
  Batch   200  of    292.
  Batch   250  of    292.

Training Loss: 0.002
Validation Loss: 0.339

 Epoch 2 / 4
  Batch    50  of    876.
  Batch   100  of    876.
  Batch   150  of    876.
  Batch   200  of    876.
  Batch   250  of    876.
  Batch   300  of    876.
  Batch   350  of    876.
  Batch   400  of    876.
  Batch   450  of    876.
  Batch   500  of    876.
  Batch   550  of    876.
  Batch   600  of    876.
  Batch   650  of    876.
  

Fine-Tuning for a time quantum of 4 months

In [ ]:
for i in range(4,28,4):
  train_text_cov=df1.loc[:int(0.0416*i*0.6*df1.shape[0]),'data'].values
  train_labels_cov=df1.loc[:int(0.0416*i*0.6*df1.shape[0]),'label'].values
  val_text_cov=df1.loc[int(0.0416*i*0.6*df1.shape[0]):int(0.0416*i*0.8*df1.shape[0]),'data'].values
  val_labels_cov=df1.loc[int(0.0416*i*0.6*df1.shape[0]):int(0.0416*i*0.8*df1.shape[0]),"label"].values
  test_text_cov=df1.loc[int(0.0416*i*0.8*df1.shape[0]):,'data'].values
  test_labels_cov=df1.loc[int(0.0416*i*0.8*df1.shape[0]):,'label'].values
  max_seq_len = 15
  tokens_train = tokenizer.batch_encode_plus(
      train_text_cov.tolist(),
      max_length = max_seq_len,
      pad_to_max_length=True,
      truncation=True,
      return_token_type_ids=False
  )
  tokens_val = tokenizer.batch_encode_plus(
    val_text_cov.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)
  tokens_test = tokenizer.batch_encode_plus(
      test_text_cov.tolist(),
      max_length = max_seq_len,
      pad_to_max_length=True,
      truncation=True,
      return_token_type_ids=False
  )
  train_seq = torch.tensor(tokens_train['input_ids'])
  train_mask = torch.tensor(tokens_train['attention_mask'])
  train_y = torch.tensor(train_labels_cov.tolist())
  val_seq = torch.tensor(tokens_val['input_ids'])
  val_mask = torch.tensor(tokens_val['attention_mask'])
  val_y = torch.tensor(val_labels_cov.tolist())
  test_seq = torch.tensor(tokens_test['input_ids'])
  test_mask = torch.tensor(tokens_test['attention_mask'])
  test_y = torch.tensor(test_labels_cov.tolist())
  batch_size = 12
  train_data = TensorDataset(train_seq, train_mask, train_y)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
  val_data = TensorDataset(val_seq, val_mask, val_y)
  val_sampler = SequentialSampler(val_data)
  val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)
  best_valid_loss = float('inf')
  train_losses=[]
  valid_losses=[]
  epochs=4
  for epoch in range(epochs):
      print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
      train_loss, _ = train()
      valid_loss, _ = evaluate()
      if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model4.state_dict(), 'saved_weights4.pt')
      train_losses.append(train_loss)
      valid_losses.append(valid_loss)
      print(f'\nTraining Loss: {train_loss:.3f}')
      print(f'Validation Loss: {valid_loss:.3f}')
  path = 'saved_weights4.pt'
  model4.load_state_dict(torch.load(path))
  with torch.no_grad():
    preds,_ = model4(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()
  preds = np.argmax(preds, axis = 1)
  test_y = torch.tensor(test_labels_cov.tolist())
  print(classification_report(test_y, preds))

Fine-Tuning for a time quantum of 1 month